#### Copyright 2020 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Polynomial Regression and Overfitting

So far in this course, we have dealt exclusively with linear models. These have all been "straight-line" models, where we attempt to draw a straight line that fits a regression.

Today, we will start building curved-lined models based on [polynomial equations](https://en.wikipedia.org/wiki/Polynomial).

## Generating Sample Data

Let's start by generating some data based on a second degree polynomial.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

num_items = 100

np.random.seed(seed=420)
X = np.random.randn(num_items, 1)

# These coefficients are chosen arbitrarily.
y = 0.6*(X**2) - 0.4*X + 1.3

plt.plot(X, y, 'b.')
plt.show()

Let's add some randomness to create a more realistic dataset, and re-plot the randomized data points and the fit line.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

num_items = 100

np.random.seed(seed=420)
X = np.random.randn(num_items, 1)

# Create some randomness.
randomness = np.random.randn(num_items, 1) / 2

# This is the same equation as the plot above, with added randomness.
y = 0.6*(X**2) - 0.4*X + 1.3 + randomness

X_line = np.linspace(X.min(), X.max(), num=num_items)
y_line = 0.6*(X_line**2) - 0.4*X_line + 1.3

plt.plot(X, y, 'b.')
plt.plot(X_line, y_line, 'r-')
plt.show()

That looks much better! Now we can see that a 2-degree polynomial function fits this data reasonably well.

## Polynomial Fitting

We can now see a pretty obvious 2-degree polynomial that fits the scatterplot.

Scikit-learn offers a `PolynomialFeatures` class that handles polynomial combinations for a linear model. In this case, we know that a 2-degree polynomial is a good fit, since the data was generated from a polynomial curve. Let's see if the model works.

We begin by creating a `PolynomialFeatures` instance of degree 2.

In [0]:
from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=2)
X_poly = pf.fit_transform(X)

X.shape, X_poly.shape

You might be wondering what the `include_bias` parameter is. By default, it is `True`, in which case it forces the first exponent to be 0.

This adds a constant bias term to the equation. When we ask for no bias we start our exponents at 1 instead of 0.

This preprocessor generates a new feature matrix consisting of all polynomial combinations of the features. Notice that the input shape of `(100, 1)` becomes `(100, 2)` after transformation.

In this simple case, we doubled the number of features since we asked for a 2-degree polynomial and had one input feature. The number of generated features grows exponentially as the number of features and polynomial degrees increases.

## Model Fitting

We can now fit the model by passing our polynomial preprocessing data to the linear regressor.

How close did the intercept and coefficient match the values in the function we used to generate our data?

In [0]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)

lin_reg.intercept_, lin_reg.coef_

## Visualization

We can plot our fitted line against the equation we used to generate the data. The fitted line is green and the actual curve is red.

In [0]:
np.random.seed(seed=420)

X_line_fitted = np.linspace(X.min(), X.max(), num=100)
y_line_fitted = lin_reg.coef_[0][1] * X_line_fitted**2 + lin_reg.coef_[0][0] * X_line_fitted + lin_reg.intercept_

plt.plot(X_line_fitted, y_line_fitted, 'g-')
plt.plot(X_line, y_line, 'r-')
plt.plot(X, y, 'b.')
plt.show()

# Overfitting

When using polynomial regression, it can be easy to *overfit* the data so that it performs well on the training data, but doesn't perform well in the real world.

To understand overfitting, we will create a fake dataset generated off of a linear equation, but use a polynomial regression as the model.

In [0]:
np.random.seed(seed=420)

# Create 50 points from a linear dataset with randomness.
num_items = 50
X = 6 * np.random.rand(num_items, 1)
y = X + 2 + np.random.randn(num_items, 1)

X_line = np.array([X.min(), X.max()])
y_line = X_line + 2

plt.plot(X_line, y_line, 'r-')
plt.plot(X, y, 'b.')
plt.show()

Let's now create a 10 degree polynomial to fit the linear data and fit the model.

In [0]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

np.random.seed(seed=420)

poly_features = PolynomialFeatures(degree=10, include_bias=False)
X_poly = poly_features.fit_transform(X)

regression = LinearRegression()
regression.fit(X_poly, y)

## Visualization

Let's draw the polynomial line that we fit to the data. To draw the line, we need to execute the 10 degree polynomial equation.

$$
y = k_0 + k_1x^1 + k_2x^2 + k_3x^3 + ... + k_9x^9 + k_{10}x^{10}
$$

Coding the above equation by hand is tedious and error-prone. It also makes it difficult to change the degree of the polynomial we are fitting.

Let's see see if there is a way to write the code more dynamically, using the `PolynomialFeatures` and `LinearRegression` functions.

The `PolynomialFeatures` class provides us with a list of exponents that we can use for each portion of the polynomial equation.

In [0]:
poly_features.powers_

The `LinearRegression` class provides us with a list of coefficients that correspond to the powers provided by `PolynomialFeatures`.

In [0]:
regression.coef_

It also provides an intercept.

In [0]:
regression.intercept_

Having this information, we can take a set of $X$ values (in the code below we use 100), then run our equation on those values.

In [0]:
np.random.seed(seed=420)

# Create 100 even-spaced x-values.
X_line_fitted = np.linspace(X.min(), X.max(), num=100)

# Start our equation with the intercept.
y_line_fitted = regression.intercept_

# For each exponent, raise the X value to that exponent and multiply it by the
# appropriate coefficient
for i in range(len(poly_features.powers_)):
  exponent = poly_features.powers_[i][0]
  y_line_fitted = y_line_fitted + \
    regression.coef_[0][i] * (X_line_fitted**exponent)

We can now plot the data points, the actual line used to generate them, and our fitted model.

In [0]:
plt.plot(X_line, y_line, 'r-')
plt.plot(X_line_fitted, y_line_fitted, 'g-')
plt.plot(X, y, 'b.')
plt.show()

Notice how our line is very wavy, and spikes up and down to pass through specific data points. (This is especially true for the lowest and highest $x$-values, where the curve passes through them exactly.) This is a sign of overfitting. The line fits the training data reasonably well, but may not be as useful on new data.

## Using a Simpler Model

The most obvious way to prevent overfitting in this example is to simply reduce the degree of the polynomial.

The code below uses a 2-degree polynomial and seems to fit the data much better. A linear model would work well too.

In [0]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

regression = LinearRegression()
regression.fit(X_poly, y)

X_line_fitted = np.linspace(X.min(), X.max(), num=100)
y_line_fitted = regression.intercept_
for i in range(len(poly_features.powers_)):
  exponent = poly_features.powers_[i][0]
  y_line_fitted = y_line_fitted + \
    regression.coef_[0][i] * (X_line_fitted**exponent)

plt.plot(X_line, y_line, 'r-')
plt.plot(X_line_fitted, y_line_fitted, 'g-')
plt.plot(X, y, 'b.')
plt.show()

## Lasso Regularization

It is not always so clear what the "simpler" model choice is. Often, you will have to rely on regularization methods. A **regularization** is a method that penalizes large coefficients, with the aim of shrinking unnecessary coefficients to zero.

Least Absolute Shrinkage and Selection Operator (Lasso) regularization, also called L1 regularization, is a regularization method that adds the sum of the absolute values of the coefficients as a penalty in a cost function.

In scikit-learn, we can use the [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) model, which performs a linear regression with an L1 regression penalty.

In the resultant graph, you can see that the regression smooths out our polynomial curve quite a bit, despite the polynomial being a degree 10 polynomial. Note that Lasso regression can make the impact of less important features completely disappear.

In [0]:
from sklearn.linear_model import Lasso

poly_features = PolynomialFeatures(degree=10, include_bias=False)
X_poly = poly_features.fit_transform(X)

lasso_reg = Lasso(alpha=5.0)
lasso_reg.fit(X_poly, y)

X_line_fitted = np.linspace(X.min(), X.max(), num=100)
y_line_fitted = lasso_reg.intercept_
for i in range(len(poly_features.powers_)):
  exponent = poly_features.powers_[i][0]
  y_line_fitted = y_line_fitted + lasso_reg.coef_[i] * (X_line_fitted**exponent)

plt.plot(X_line, y_line, 'r-')
plt.plot(X_line_fitted, y_line_fitted, 'g-')
plt.plot(X, y, 'b.')
plt.show()

## Ridge Regularization

Similar to Lasso regularization, [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) regularization adds a penalty to the cost function of a model. In the case of Ridge, also called L2 regularization, the penalty is the sum of squares of the coefficients.

Again, we can see that the regression smooths out the curve of our 10-degree polynomial.

In [0]:
from sklearn.linear_model import Ridge

poly_features = PolynomialFeatures(degree=10, include_bias=False)
X_poly = poly_features.fit_transform(X)

ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(X_poly, y)

X_line_fitted = np.linspace(X.min(), X.max(), num=100)
y_line_fitted = ridge_reg.intercept_
for i in range(len(poly_features.powers_)):
  exponent = poly_features.powers_[i][0]
  y_line_fitted = y_line_fitted + ridge_reg.coef_[0][i] * (X_line_fitted**exponent)

plt.plot(X_line, y_line, 'r-')
plt.plot(X_line_fitted, y_line_fitted, 'g-')
plt.plot(X, y, 'b.')
plt.show()

## ElasticNet Regularization

Another common form of regularization is [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) regularization. This regularization method combines the concepts of L1 and L2 regularization by applying a penalty containing both a squared value and an absolute value.

In [0]:
from sklearn.linear_model import ElasticNet

poly_features = PolynomialFeatures(degree=10, include_bias=False)
X_poly = poly_features.fit_transform(X)

elastic_reg = ElasticNet(alpha=2.0, l1_ratio=0.5)
elastic_reg.fit(X_poly, y)

X_line_fitted = np.linspace(X.min(), X.max(), num=100)
y_line_fitted = elastic_reg.intercept_
for i in range(len(poly_features.powers_)):
  exponent = poly_features.powers_[i][0]
  y_line_fitted = y_line_fitted + \
    elastic_reg.coef_[i] * (X_line_fitted**exponent)

plt.plot(X_line, y_line, 'r-')
plt.plot(X_line_fitted, y_line_fitted, 'g-')
plt.plot(X, y, 'b.')
plt.show()

## Other Strategies

Aside from regularization, there are other strategies that can be used to prevent overfitting. These include:

* [Early stopping](https://en.wikipedia.org/wiki/Early_stopping)
* [Cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics))
* [Ensemble methods](https://en.wikipedia.org/wiki/Ensemble_learning)
* Simplifying your model
* Removing features

# Exercises

For these exercises, we will be working with the [diabetes dataset](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset) that comes with scikit-learn. The data contains the following features:

1. age
1. sex
1. body mass index (bmi)
1. average blood pressure (bp)

It also contains six measures of blood serum, `s1` through `s6`. The target is a numeric assessment of the progression of the disease over the course of a year.

The data has been standardized.

In [0]:
from sklearn.datasets import load_diabetes

import numpy as np
import pandas as pd

data = load_diabetes()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['progression'] = data.target

df.describe()

Let's plot how body mass index relates to blood pressure.

In [0]:
import matplotlib.pyplot as plt

plt.plot(df['bmi'], df['bp'], 'b.')
plt.show()

## Exercise 1: Polynomial Regression 

Let's create a model to see if we can map body mass index to blood pressure.

1. Create a 10-degree polynomial preprocessor for our regression
1. Create a linear regression model
1. Fit and transform the `bmi` values with the polynomial features preprocessor
1. Fit the transformed data using the linear regression
1. Plot the fitted line over a scatterplot of the data points

**Student Solution**

In [0]:
# Your code goes here

---

### Answer Key

In [0]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

data = load_diabetes()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['progression'] = data.target

X = df[['bmi']]
y = df['bp']

poly_features = PolynomialFeatures(degree=10, include_bias=False)
X_poly = poly_features.fit_transform(X)

regression = LinearRegression()
regression.fit(X_poly, y)

X_line_fitted = np.linspace(X.min(), X.max(), num=100)
y_line_fitted = regression.intercept_

for i in range(len(poly_features.powers_)):
  exponent = poly_features.powers_[i][0]
  y_line_fitted = y_line_fitted + \
    regression.coef_[i] * (X_line_fitted**exponent)

plt.plot(X_line_fitted, y_line_fitted, 'g-')
plt.plot(X, y, 'b.')
plt.show()

---

## Exercise 2: Regularization

Your model from exercise one likely looked like it overfit. Experiment with the Lasso, Ridge, and/or ElasticNet classes in the place of the `LinearRegression`. Adjust the parameters for whichever regularization class you use until you create a line that doesn't look to be under- or over-fitted.

**Student Solution**

In [0]:
# Your code goes here

---

### Answer Key

In [0]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_diabetes
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

data = load_diabetes()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['progression'] = data.target

X = df[['bmi']]
y = df['bp']

poly_features = PolynomialFeatures(degree=10, include_bias=False)
X_poly = poly_features.fit_transform(X)

regression = Ridge()
regression.fit(X_poly, y)

X_line_fitted = np.linspace(X.min(), X.max(), num=100)
y_line_fitted = regression.intercept_

for i in range(len(poly_features.powers_)):
  exponent = poly_features.powers_[i][0]
  y_line_fitted = y_line_fitted + \
    regression.coef_[i] * (X_line_fitted**exponent)

plt.plot(X_line_fitted, y_line_fitted, 'g-')
plt.plot(X, y, 'b.')
plt.show()

---

## Exercise 3: Other Models

Experiment with the [BayesianRidge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html). Does it's fit line look better or worse than your other models?

**Student Solution**

In [0]:
# Your code goes here.

Does your fit line look better or worse than your other models?

> *Your Answer Goes Here*

---

### Answer Key

**Solution**

In [0]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_diabetes
from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

data = load_diabetes()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['progression'] = data.target

X = df[['bmi']]
y = df['bp']

poly_features = PolynomialFeatures(degree=10, include_bias=False)
X_poly = poly_features.fit_transform(X)

regression = BayesianRidge()
regression.fit(X_poly, y)

X_line_fitted = np.linspace(X.min(), X.max(), num=100)
y_line_fitted = regression.intercept_

for i in range(len(poly_features.powers_)):
  exponent = poly_features.powers_[i][0]
  y_line_fitted = y_line_fitted + \
    regression.coef_[i] * (X_line_fitted**exponent)

plt.plot(X_line_fitted, y_line_fitted, 'g-')
plt.plot(X, y, 'b.')
plt.show()

Does your fit line look better or worse than your other models?

> The answers are somewhat open here depending on how the students tuned the models. In our case, when we applied ridge regression we seemed to underfit a bit compared to Bayesian modelling. However, our Bayesian model might have been a little overfit for high BMI values.

---